In [2]:
from skimage import data, io
from matplotlib import pyplot as plt

def imread(file):
    try:
        img = io.imread(file+'.jpg')
    except:
        try:
            img = io.imread(file+'.jpeg')
        except:
            raise
    return img

img = imread('/furniture-data/test/1')
io.imshow(img)
plt.show()

/home/1nnuser/lsdaenv/lib/python3.5/site-packages/skimage/io/_plugins/matplotlib_plugin.py:51: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  out_of_range_float = (np.issubdtype(image.dtype, np.float) and
/home/1nnuser/lsdaenv/lib/python3.5/site-packages/matplotlib/axes/_base.py:1400: MatplotlibDeprecationWarning: The 'box-forced' keyword argument is deprecated since 2.2.
  " since 2.2.", cbook.mplDeprecation)


<Figure size 640x480 with 1 Axes>

In [3]:
# import cv2
import numpy as np
from joblib import Parallel, delayed
import concurrent.futures
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import glob, os, sys
import pickle

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import load_model

import numpy as np

/home/1nnuser/lsdaenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from keras import backend as K
# model = load_model('model.h5')
model = ResNet50(weights='imagenet', include_top=False)
# get_features_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[-3].output])
# n_samples = 12000
# progress_bar = tqdm(total=n_samples)

In [ ]:
file = '/furniture-data/test/1.jpg'
print(file)
img = image.load_img(file, target_size=(224, 224))
# print(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
# layer_output = get_features_layer_output([x])[0]
#     progress_bar.update(multiply)
model.predict(x)

/furniture-data/test/1.jpg


In [ ]:
def get_features(file, multiply):
    print(file)
    img = image.load_img(file, target_size=(224, 224))
    # print(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    # layer_output = get_features_layer_output([x])[0]
#     progress_bar.update(multiply)
    return [file, model.predict(x).reshape(-1)]

def extract_features(mode):
    data_path = '/furniture-data/'
    pickle_name_core = data_path + 'features/cnn_features_'+mode
    

    if mode == 'test':
        pickle_name = pickle_name_core + '.pkl'
        try:
            dict_prev = pickle.load(open(pickle_name, 'rb'))
        except:
            dict_prev = {}
        dict_len = len(dict_prev.keys())
        
        if(dict_prev):
            print(pickle_name, " already exists.")
            return
        else:
            print("Extracting features for: ", mode)

        files = glob.glob(data_path + mode + "/*")
        print(files)
#         progress_bar.total = len(files)
        n_jobs = 4
        print("FILES:", len(files))
        X = Parallel(n_jobs=n_jobs)(delayed(get_features)(file, n_jobs) for file in files)
        print("len of X", len(X))
        
        if(len(X) == 0):
            raise
        # if(len(X) != n_samples)
        #     n_iters += 1
        # print("--- %s seconds ---" % (time.time() - start_time))
        dict_new = {item[0]: item[1] for item in X}

        with open(pickle_name, 'wb') as f:
            pickle.dump(dict_new, f)
        return
        
    for batch_i in range(1,129):
        
        pickle_name = pickle_name_core + '_' + str(batch_i) + '.pkl'
        try:
            dict_prev = pickle.load(open(pickle_name, 'rb'))
        except:
            dict_prev = {}
        dict_len = len(dict_prev.keys())
        
        if(dict_prev):
            print(pickle_name, " already exists.")
            continue
        else:
            print("Extracting features for: ", mode, batch_i)
        
        X = list()
        start_time = time.time()
        files = glob.glob(data_path + mode + "/" + str(batch_i) + "/*")
#         progress_bar.total = len(files)
        [X.append(get_features(file, 1)) for file in files]
#         n_jobs = 4
        print("FILES:", len(files))
#         X = Parallel(n_jobs=n_jobs)(delayed(get_features)(file, n_jobs) for file in files)
        print("len of X", len(X))
        
        if(len(X) == 0):
            raise
        # if(len(X) != n_samples)
        #     n_iters += 1
        print("--- %s seconds ---" % (time.time() - start_time))
        dict_new = {item[0]: item[1] for item in X}

        with open(pickle_name, 'wb') as f:
            pickle.dump(dict_new, f)
        
    # X = pickle.load(open('hist_features.pkl', 'rb'))
    # print(len(X))

In [ ]:
extract_features('train')